In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#create dataloaders for training
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

# Define paths to stored pickle files
DATASET_PATH = "/content/drive/My Drive/Colab Notebooks/APS360-project/Chord_Pandas_Dataset_specto"
train_pkl = os.path.join(DATASET_PATH, "train.pkl")
val_pkl = os.path.join(DATASET_PATH, "val.pkl")
test_pkl = os.path.join(DATASET_PATH, "test.pkl")

# Load datasets from pickle files
df_train = pd.read_pickle(train_pkl)
df_val = pd.read_pickle(val_pkl)
df_test = pd.read_pickle(test_pkl)

# Encode labels into integers
label_encoder = LabelEncoder()
df_train["label"] = label_encoder.fit_transform(df_train["label"])
df_val["label"] = label_encoder.transform(df_val["label"])
df_test["label"] = label_encoder.transform(df_test["label"])

class ChromaDatasetResNet(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #expect a (12, T) shape
        chroma = self.df.iloc[idx]["pcp"][0]
        label = self.df.iloc[idx]["label"]

        #convert chromagram to image
        #scale to [0, 255] and convert to unit8
        img = (np.clip(chroma, 0, 1) * 255).astype(np.uint8)
        #transpose so time is the horiztontal component
        chroma_img = Image.fromarray(img.T)
        chroma_img = chroma_img.convert("RGB") #grayscale -> colour

        if self.transform:
            chroma_img = self.transform(chroma_img)

        return chroma_img, label

# Create ImageNet style images by defining transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create datasets and Dataloaders
train_dataset_resnet = ChromaDatasetResNet(df_train, transform=data_transforms['train'])
val_dataset_resnet   = ChromaDatasetResNet(df_val, transform=data_transforms['val'])
test_dataset_resnet  = ChromaDatasetResNet(df_test, transform=data_transforms['val'])

train_loader_resnet = DataLoader(train_dataset_resnet, batch_size=32, shuffle=True, num_workers=2)
val_loader_resnet   = DataLoader(val_dataset_resnet, batch_size=32, shuffle=False, num_workers=2)
test_loader_resnet  = DataLoader(test_dataset_resnet, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
classes_num = len(train_dataset_resnet.df["label"].unique())

#load pretrained ResNet model
resnet_model = models.resnet18(pretrained=True)

#freeze fall layers first and then unfreeze layer 4 and fc
for name, param in resnet_model.named_parameters():
    #freeze all
    param.requires_grad = False
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

#replace the last FC layer to output the chord classes
in_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(in_features, classes_num)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = resnet_model.to(device)

In [ ]:
import time

def train_model_resnet(model, train_loader, val_loader, num_epochs, criterion, optimizer):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  start_time = time.time()
  train_acc_history = np.zeros(num_epochs)
  val_acc_history = np.zeros(num_epochs)
  train_loss_history = np.zeros(num_epochs)
  val_loss_history = np.zeros(num_epochs)

  print("Starting Training")
  for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0

    for features, labels in train_loader:
      features, labels = features.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(features)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      train_correct += (outputs.argmax(1) == labels).sum().item()

    train_loss_history[epoch] = train_loss/ len(train_loader)
    train_acc_history[epoch] = train_correct / len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
      for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        val_correct += (outputs.argmax(1) == labels).sum().item()

    val_loss_history[epoch] = val_loss / len(val_loader)
    val_acc_history[epoch] = val_correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss_history[epoch]:.4f}, Train Acc: {train_acc_history[epoch]:.4f}, || Val Loss: {val_loss_history[epoch]:.4f}, Val Acc: {val_acc_history[epoch]:.4f}")

  print("Finished Training")
  end_time = time.time()
  print(f"Training time: {end_time - start_time:.2f} seconds")

  #save training files
  path = "/content/drive/My Drive/Colab Notebooks/APS360-project/resnet"
  np.savetxt("{}train_acc.csv".format(path), train_acc_history)
  np.savetxt("{}val_acc.csv".format(path), val_acc_history)
  np.savetxt("{}train_loss.csv".format(path), train_loss_history)
  np.savetxt("{}val_loss.csv".format(path), val_loss_history)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {"params": resnet_model.layer4.parameters(), "lr": 0.0001}, #lower learning rate for tuning layer 4
    {"params": resnet_model.fc.parameters(), "lr": 0.001} #higher learning rate for final layer
])

train_model_resnet(resnet_model, train_loader_resnet, val_loader_resnet, 40, criterion, optimizer)

NameError: name 'nn' is not defined

In [ ]:
#test model function
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    test_correct = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = test_correct / len(test_loader.dataset)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [ ]:
#testing accuracy
criterion = nn.CrossEntropyLoss()
test_model(resnet_model, test_loader_resnet, criterion)